# 1. Story_Canvas

# (1) 

In [3]:
import transformers
import requests
import PIL.Image
import io

In [4]:
# 1. Hugging Face의 free train 모델을 가져와서 기존의 소설을 학습한다.
free_train_model = transformers.pipeline("text-generation", model="EleutherAI/gpt-neo-1.3B")

In [5]:
# 2. Hugging Face의 free train, api를 활용하여 소설속 삽화를 위한 이미지를 학습한다.
# 이미지 생성을 위한 API의 엔드포인트 URL입니다.
API_ENDPOINT = "https://api-inference.huggingface.co/models/vdbayar/gan-paint"


# 이미지 생성을 위한 함수
def generate_image(text, model_name, api_endpoint):
    # 모델과 API에 대한 요청을 만듦
    model = transformers.AutoModelForImage2TextLM.from_pretrained(model_name)
    input_dict = {
        "inputs": text,
        "parameters": {
            "max_new_images": 1,
            "size": "512x512"
        }
    }
    headers = {"Authorization": f"Bearer {api_endpoint}"}
    response = requests.post(API_ENDPOINT, json=input_dict, headers=headers)

    # 반환된 이미지를 Pillow 객체로 변환
    img_bytes = io.BytesIO(response.content)
    img = PIL.Image.open(img_bytes)
    return img

In [6]:
# 3. 사용자로부터 선호하는 작가의 스타일과, 주인공의 이름, 선호하는 그림 스타일을 입력 받는다.
author_style = input("선호하는 작가의 스타일을 입력하세요: ")
protagonist_name = input("주인공의 이름을 입력하세요: ")
image_style = input("선호하는 그림 스타일을 입력하세요: ")

선호하는 작가의 스타일을 입력하세요: Antoine de Saint-Exupéry
주인공의 이름을 입력하세요: Moon
선호하는 그림 스타일을 입력하세요: Vincent van Gogh


In [7]:
# 4. 사용자로부터 첫 줄을 입력 받는다.
user_input = input("첫 줄을 입력하세요: ")

첫 줄을 입력하세요: I decided to travel around the world on the morning of my twentieth birthday.


In [8]:
# 5. 입력에 대한 단편 소설을 생성한다.
novel = free_train_model(user_input, max_length=1000, num_return_sequences=1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [9]:
# 6. 생성된 단편 소설을 10개로 나눈다.
novel_sentences = novel[0]['generated_text'].split(". ")
novel_sentences = [sentence.strip() for sentence in novel_sentences]

In [10]:
# 7. 10개 각각을 요약한다.
summary_sentences = []
for sentence in novel_sentences:
    summary = free_train_model(sentence, max_length=50, num_return_sequences=1)
    summary_sentences.append(summary[0]['generated_text'].strip())

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 55, but ``max_length`` is set to 50.This can lead to unexpected behavior. You should consider increasing ``config.max_length`` or ``max_length``.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_to

In [11]:
# 8. 요약된 문장에 대해 이미지를 생성하기 위해 GAN 모델로 이미지를 생성한다.
model_name = "vdbayar/gan-paint"
generated_images = []
for sentence in summary_sentences:
    image = generate_image(f"{author_style} 스타일의 {protagonist_name}가 {sentence}", model_name, image_style)
    generated_images.append(image)

AttributeError: module transformers has no attribute AutoModelForImage2TextLM

In [12]:
# 9. 10개로 나눠진 소설과, 해당 요약에 대한 이미지를 출력한다.
for i in range(10):
    print(f"소설 {i+1}: {novel_sentences[i]}'")
#     generated_images[i].show()

소설 1: I decided to travel around the world on the morning of my twentieth birthday'
소설 2: As it turned out I was in a pretty good mood and it took me an hour and a half to drive to the small town of Ithaca, NY'
소설 3: I went there with the intention of taking a trip of a lifetime'
소설 4: At the beginning of the trip I would have been perfectly happy to spend the day driving around Ithaca to see the sites that would have been a part of my childhood'
소설 5: But as it turned out it was much more fun to see how the town looked from the outside from the car in the car in the car and in the car'
소설 6: I was also lucky enough to have a great view from my window of the beautiful Cornell University campus.

After driving around for a while I was pretty sure that I should come to a point at which I would be able to park the car and go look at the buildings'
소설 7: I had already rented the car on vacation and I knew the price of admission to any of the buildings was quite reasonable'
소설 8: I didn’t k

In [13]:
print(novel)

[{'generated_text': 'I decided to travel around the world on the morning of my twentieth birthday. As it turned out I was in a pretty good mood and it took me an hour and a half to drive to the small town of Ithaca, NY. I went there with the intention of taking a trip of a lifetime. At the beginning of the trip I would have been perfectly happy to spend the day driving around Ithaca to see the sites that would have been a part of my childhood. But as it turned out it was much more fun to see how the town looked from the outside from the car in the car in the car and in the car. I was also lucky enough to have a great view from my window of the beautiful Cornell University campus.\n\nAfter driving around for a while I was pretty sure that I should come to a point at which I would be able to park the car and go look at the buildings. I had already rented the car on vacation and I knew the price of admission to any of the buildings was quite reasonable. I didn’t know the exact height of t